In [1]:
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression, Ridge, Lasso 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from scipy.signal import find_peaks

In [2]:
# opening the dataset
f = open('./dataset_OSAS.pickle', 'rb')
data = pickle.load(f)
f.close()

In [3]:
# 1600 in one 20 second window 
# finding windows, extracting features 

patients = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '2',
       '19', '20', '21', '22', '23', '24', '25', '26', '3', '27', '28',
       '29', '30', '4', '5', '6', '7', '8', '9']

smaller_event = []

feature_dict = {}

features = ['PSG_Flow', 'PSG_Thorax', 'PSG_Abdomen', 'signal_ecg_i', 'signal_ecg_ii']
feature_dict['y'] = []
feature_dict['patient'] = []

for f in features:
    feature_dict[f+'_std'] = []
    feature_dict[f+'_rms'] = []
    feature_dict[f+'_med'] = []
    for pt in patients:

        patient = data[data['patient'] == pt]

        psg = list(patient[f])
        big_arr = np.array(psg).flatten()
        event_arr = list(patient['event'])
        ev_arr = np.array(event_arr).flatten()

        start = 0
        
        if f == 'PSG_Flow':
            high_window = 400
            prom = 0.5
            add = 200
            
        elif f == 'PSG_Thorax':
            high_window = 200
            prom = 0.1
            add = 100
        
        elif f == 'PSG_Abdomen':
            high_window = 200
            prom = 0.2
            add = 100
            
        elif f == 'PSG_Snore':
            high_window = 200
            thresh = 1
            dist = 10
            add = 100
        
        else:
            # signal_ecg_i and signal_ecg_ii
            high_window = 1600
            thresh = 2.2
            add = 800
            
        
        while start + high_window < len(big_arr):
            if f == 'PSG_Snore':
                interval = find_peaks(big_arr[start:start + high_window], threshold = thresh, distance = dist)[0]
            elif high_window != 800:
                interval = find_peaks(big_arr[start:start + high_window], prominence = prom)[0]
            else:
                interval = find_peaks(big_arr[start:start + high_window], threshold = thresh)[0]
            feature_dict[f+'_std'] = np.append(feature_dict[f+'_std'], np.std(interval))
            feature_dict[f+'_rms'] = np.append(feature_dict[f+'_rms'], np.sqrt(np.mean(interval**2)))
            feature_dict[f+'_med'] = np.append(feature_dict[f+'_med'], np.median(interval))
            start += add

        if f == 'PSG_Flow':
            start_ev = 0
            while start_ev + 20 < len(ev_arr):
                feature_dict['patient'] = np.append(feature_dict['patient'], pt)
                ev_apnea = 'APNEA-CENTRAL' in ev_arr[start_ev:start_ev+20]
                if ev_apnea == False:
                    ev_apnea = 'APNEA-MIXED' in ev_arr[start_ev:start_ev+20]
                    if ev_apnea == False:
                        ev_apnea = 'APNEA-OBSTRUCTIVE' in ev_arr[start_ev:start_ev+20]
                ev_hypopnea = 'HYPOPNEA' in ev_arr[start_ev:start_ev+20]
                if ev_apnea == True:
                    feature_dict['y'] = np.append(feature_dict['y'], 1)
                elif ev_hypopnea == True:
                    feature_dict['y'] = np.append(feature_dict['y'], 2)
                else:
                    feature_dict['y'] = np.append(feature_dict['y'], 0)
                start_ev += 10



/Users/piperfrances/miniconda3/envs/ds4h/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/piperfrances/miniconda3/envs/ds4h/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/piperfrances/miniconda3/envs/ds4h/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/piperfrances/miniconda3/envs/ds4h/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/piperfrances/miniconda3/envs/ds4h/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret 

In [7]:
# make it into a pandas dataframe
df = pd.DataFrame.from_dict(feature_dict)

In [99]:
#df.to_pickle("psg_ecg_all_features.pkl")

In [8]:
# remove NaNs from df 

heads = ['PSG_Flow_std','PSG_Flow_rms','PSG_Flow_med',
                       'PSG_Thorax_std','PSG_Thorax_rms', 'PSG_Thorax_med',
                       'PSG_Abdomen_std', 'PSG_Abdomen_rms', 'PSG_Abdomen_med',
                       'signal_ecg_i_std', 'signal_ecg_i_rms', 'signal_ecg_i_med',
                       'signal_ecg_ii_std', 'signal_ecg_ii_rms', 'signal_ecg_ii_med',
                       #'PSG_Snore_std','PSG_Snore_rms', 'PSG_Snore_med'
        ]

psg_no_ecg = ['PSG_Flow_std','PSG_Flow_rms', 'PSG_Flow_med',
                       'PSG_Thorax_std','PSG_Thorax_rms', 'PSG_Thorax_med',
                       'PSG_Abdomen_std', 'PSG_Abdomen_rms', 'PSG_Abdomen_med',
                       #'PSG_Snore_std','PSG_Snore_rms', 'PSG_Snore_med'
             ]

ecg_only = ['signal_ecg_i_std', 'signal_ecg_i_rms', 'signal_ecg_i_med',
                       'signal_ecg_ii_std', 'signal_ecg_ii_rms', 'signal_ecg_ii_med']

psg_rms = ['PSG_Flow_rms',
                       'PSG_Thorax_rms',
                       'PSG_Abdomen_rms',
                       #'PSG_Snore_rms'
          ]

psg_std = ['PSG_Flow_std',
                       'PSG_Thorax_std',
                       'PSG_Abdomen_std',
                       #'PSG_Snore_std'
          ]

psg_med = ['PSG_Flow_med',
                       'PSG_Thorax_med',
                       'PSG_Abdomen_med',
                       #'PSG_Snore_med'
          ]

label_array = [heads, psg_no_ecg, ecg_only, psg_rms, psg_std, psg_med]

df = df.dropna(subset=heads)
subset = df[heads]
x = subset.to_numpy()
y = df['y'].to_numpy()


## 3 Class Logistic Regression 
### 'NONE', 'APNEA', 'HYPOPNEA'

In [9]:
# training on: heads, psg_no_ecg, ecg_only, psg_rms psg_std, psg_med
tests = ['all', 'psg only', 'ecg only', 'psg root mean square', 'psg standard deviation', 'psg median']

from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve, roc_auc_score

for labels in range(len(label_array)):

    subset = df[label_array[labels]]
    reshape = np.shape(subset)[1]

    x = subset.to_numpy()
    y = df['y'].to_numpy()

    oversample = SMOTE()
    #x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    x, y = oversample.fit_resample(x, y)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8, random_state = 0)

    logModel = LogisticRegression(multi_class = 'multinomial')
    x_train = x_train.reshape(-1, reshape)
    x_test = x_test.reshape(-1, reshape)
    logModel.fit(x_train, y_train)
    y_pred = logModel.predict(x_test)
    mean_squared_error(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['None', 'APNEA', 'HYPOPNEA'])
    print(tests[labels])
    print(report)

/Users/piperfrances/miniconda3/envs/ds4h/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


all
              precision    recall  f1-score   support

        None       0.49      0.52      0.51      9164
       APNEA       0.49      0.46      0.48      9239
    HYPOPNEA       0.47      0.47      0.47      9221

    accuracy                           0.48     27624
   macro avg       0.48      0.48      0.48     27624
weighted avg       0.48      0.48      0.48     27624

psg only
              precision    recall  f1-score   support

        None       0.50      0.59      0.54      9164
       APNEA       0.48      0.32      0.38      9239
    HYPOPNEA       0.49      0.57      0.52      9221

    accuracy                           0.49     27624
   macro avg       0.49      0.49      0.48     27624
weighted avg       0.49      0.49      0.48     27624



/Users/piperfrances/miniconda3/envs/ds4h/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ecg only
              precision    recall  f1-score   support

        None       0.47      0.58      0.52      9164
       APNEA       0.45      0.44      0.45      9239
    HYPOPNEA       0.37      0.30      0.33      9221

    accuracy                           0.44     27624
   macro avg       0.43      0.44      0.43     27624
weighted avg       0.43      0.44      0.43     27624

psg root mean square
              precision    recall  f1-score   support

        None       0.39      0.33      0.36      9164
       APNEA       0.47      0.39      0.43      9239
    HYPOPNEA       0.44      0.58      0.50      9221

    accuracy                           0.43     27624
   macro avg       0.43      0.43      0.43     27624
weighted avg       0.43      0.43      0.43     27624

psg standard deviation
              precision    recall  f1-score   support

        None       0.44      0.65      0.53      9164
       APNEA       0.47      0.26      0.33      9239
    HYPOPNEA       0.4

## Now 2 Classes

In [10]:
# training on: heads, psg_no_ecg, ecg_only, psg_rms psg_std, psg_med
tests = ['all', 'psg only', 'ecg only', 'psg root mean square', 'psg standard deviation', 'psg median']

from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_squared_error
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

df.loc[df['y'] == 2 , 'y'] = 1

for labels in range(len(label_array)):

    subset = df[label_array[labels]]
    reshape = np.shape(subset)[1]

    x = subset.to_numpy()
    y = df['y'].to_numpy()

    oversample = SMOTE()
    #x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    x, y = oversample.fit_resample(x, y)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8, random_state = 0)

    logModel = LogisticRegression(multi_class = 'ovr')
    x_train = x_train.reshape(-1, reshape)
    x_test = x_test.reshape(-1, reshape)
    logModel.fit(x_train, y_train)
    y_pred = logModel.predict(x_test)
    mean_squared_error(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=['None', 'APNEA'])
    print(tests[labels])
    print(report)
    
#     y_predicted = logModel.predict(x)
#     y_filtered = []
#     i = 0
#     while i < len(y_predicted):
#         y_filtered.append(y_predicted[i])
#         i = i + 1
    
#     fpr, tpr, thresholds = roc_curve(y, y_filtered)
#     auc = round(roc_auc_score(y, y_filtered), 2)
#     plt.plot(fpr, tpr, '-', label = "feature = " + tests[labels]+", AUC = "+str(auc))
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.title("ROC Curve")
#     plt.legend(bbox_to_anchor=(1,0), loc="lower left")

# Learn to predict each class against the other
#     classifier = OneVsRestClassifier(
#         svm.SVC(kernel="linear", probability=True)
#     )
#     y_score = classifier.fit(x_train, y_train).decision_function(x_test)

#     # Compute micro-average ROC curve and ROC area
#     y_pred = logisticModel.predict_proba(x)
#     y_filtered = []
#     i = 0
#     while i < len(y_pred):
#         y_filtered.append(y_pred[i][1])
#         i = i + 1
#     plt.plot(fpr, tpr, '-', label = "feautre = " + str(tests[labels]))
    
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.title("ROC Curve")
#     plt.legend(loc="lower right")
#     plt.show()

/Users/piperfrances/miniconda3/envs/ds4h/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


all
              precision    recall  f1-score   support

        None       0.66      0.69      0.67      9193
       APNEA       0.68      0.64      0.66      9223

    accuracy                           0.67     18416
   macro avg       0.67      0.67      0.66     18416
weighted avg       0.67      0.67      0.66     18416

psg only
              precision    recall  f1-score   support

        None       0.64      0.70      0.67      9193
       APNEA       0.67      0.61      0.64      9223

    accuracy                           0.65     18416
   macro avg       0.66      0.65      0.65     18416
weighted avg       0.66      0.65      0.65     18416

ecg only
              precision    recall  f1-score   support

        None       0.63      0.57      0.60      9193
       APNEA       0.61      0.66      0.63      9223

    accuracy                           0.62     18416
   macro avg       0.62      0.62      0.62     18416
weighted avg       0.62      0.62      0.62     1841

In [40]:
# visualization tool! 
patient = data[data['patient'] == '1']
feature = 'PSG_Abdomen'
ecg = list(patient[feature])
big_arr = np.array(ecg).flatten()
ev_arr = np.array(patient['event'])

y = big_arr[56900: 57220]
indices = find_peaks(y, prominence = 0.2)[0]
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=y,
    mode='lines+markers',
    name=feature
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[y[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks'
))
fig.add_trace(go.Scatter(x=[92,92,280,280,92], y=[-0.75,1,1,-0.75,-0.75],name='Apnea Event'))

fig.show()